In [ ]:
# Copyright : Ryoungseob Kwon
# Cite this : Enabling 30 m maize mapping without ground reference leveraging multiple satellite platforms in Brazil and Argentina

import ee
ee.Authenticate()
ee.Initialize()

!pip install geemap -q
import geemap

import os
import numpy as np
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
target_year = 2021
target_months = np.arange(1, 13)
number_of_points = ee.Number(10000)

In [ ]:
# GEDI Functions
def qualityMask(im):
  im_masked = im.updateMask(im.select('quality_flag').eq(1)).updateMask(im.select('degrade_flag').eq(0)).updateMask(im.select('rh95').lte(5))
  return im_masked

# Filter out RH95 greater than 5 m
# Reference : https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2023EF003590

def sensitivity_filter(img):
  img_sens = img.updateMask(img.select('sensitivity').gte(0.95))
  return img_sens

# Train datasets Functions
def create_feature(i):
  return ee.Feature(i.geometry(), {'id': i.id()})

In [ ]:
# Define the function to process each crop
def process_gedi_crop(crop_name, gedi_variable, number_of_points, gedi_masked, month):

    # Use the image footprint as the region
    region = gedi_variable.geometry()

    # Perform stratified sampling
    crop_samples = gedi_variable.stratifiedSample(
        numPoints=number_of_points, scale=25, region=region, geometries=True
    )

    # Sample the GEDI data at these points, extracting all bands
    crop_features = gedi_masked.sampleRegions(
        collection=crop_samples, scale=25, tileScale=16
    )

    # Set up the export task
    task = ee.batch.Export.table.toDrive(
        collection=crop_features,  # crop_samples or crop_features
        folder=f'{month}',
        description=f'{crop_name}_GEDI_features_Month_{month}',
        fileNamePrefix=f'{crop_name}_GEDI_features_Month_{month}',
        fileFormat='CSV'    # CSV or KML
    )

    # Start the export task
    task.start()

# Process GEDI data for each month
for month in target_months:
    print(f"Processing GEDI data for Month {month}...")

    # Define time range (handle December separately)
    begin_date = ee.Date(f'{target_year}-{month:02d}-01')
    end_date   = ee.Date(f'{target_year}-{month+1:02d}-01') if month < 12 else ee.Date(f'{target_year+1}-01-01')

    # print(f'{target_year}-{month:02d}-01', f'{target_year}-{month+1:02d}-01')

    cdl_begin_date = ee.Date(f'{target_year}-01-01')
    cdl_end_date   = ee.Date(f'{target_year+1}-01-01')

    cdl = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(cdl_begin_date, cdl_end_date)).first()
    cropLandcover = cdl.select('cropland')
    cdl_composite = cropLandcover.reduce(ee.Reducer.mode())

    # Crop class masking
    corn = cdl_composite.eq(1)               # Corn
    cotton = cdl_composite.eq(2)             # Cotton
    rice = cdl_composite.eq(3)               # Rice
    sorghum = cdl_composite.eq(4)            # Sorghum
    soybean = cdl_composite.eq(5)            # Soybean
    spring_wheat = cdl_composite.eq(23)      # Spring Wheat
    winter_wheat = cdl_composite.eq(24)      # Winter Wheat

    # Reference : https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL#bands

    # Load and filter GEDI data
    gedi = (
        ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY')
        .filter(ee.Filter.date(begin_date, end_date))
        .filter(ee.Filter.calendarRange(ee.Number(int(month)), ee.Number(int(month)), 'month'))  # FIXED
        .map(qualityMask)
        .map(sensitivity_filter)
        .select('rh.*')  # Select all RH bands
    )

    # Compute mean of selected data
    gedi_mean = gedi.mean()

    # Apply a mask using RH98
    gedi_98 = gedi_mean.select('rh98')
    gedi_mask = gedi_98.gt(0)
    gedi_masked = gedi_mean.updateMask(gedi_mask)

    # Apply the mask to each crop dataset
    gedi_crops = {
        "corn": corn.updateMask(gedi_mask).selfMask(),
        "cotton": cotton.updateMask(gedi_mask).selfMask(),
        "rice": rice.updateMask(gedi_mask).selfMask(),
        "sorghum": sorghum.updateMask(gedi_mask).selfMask(),
        "soybean": soybean.updateMask(gedi_mask).selfMask(),
        "swheat": spring_wheat.updateMask(gedi_mask).selfMask(),
        "wwheat": winter_wheat.updateMask(gedi_mask).selfMask(),
    }

    # Process all crops without predefined ROI, using image footprint as the boundary
    for crop_name, gedi_variable in gedi_crops.items():
        print(f" - Processing {crop_name} for Month {month}")
        process_gedi_crop(crop_name, gedi_variable, number_of_points, gedi_masked, month)

print("Export tasks started for all crops and months.")

**Only maize vs. non-maize in train site.**

In [ ]:
number_of_points = ee.Number(25000)
target_months = [7, 8, 9]

# Train area
input_train_dir = '/content/drive/MyDrive/shapefiles'
input_train = os.path.join(input_train_dir, 'USA_train_area.shp')
input_train_ee = geemap.shp_to_ee(input_train)

# Export USA_train_area as KML
task_train_kml = ee.batch.Export.table.toDrive(
    collection=input_train_ee,
    folder='USA_train_area_export',
    description='USA_train_area_KML',
    fileNamePrefix='USA_train_area',
    fileFormat='KML'
)
task_train_kml.start()

roi = ee.FeatureCollection.geometry(input_train_ee)

# Define the function to process each crop
def process_gedi_corn(crop_name, gedi_variable, number_of_points, gedi_masked, month):

    # Perform stratified sampling
    crop_samples = gedi_variable.stratifiedSample(numPoints=number_of_points, scale=10, region=roi, geometries=True)
    crop_features = gedi_masked.sampleRegions(collection=crop_samples, scale=25, tileScale=16)

    # Set up the export task
    task = ee.batch.Export.table.toDrive(
        collection=crop_samples,    # crop_samples or crop_features
        folder=f'{month}',
        description=f'{crop_name}_GEDI_features_Month_{month}',
        fileNamePrefix=f'{crop_name}_GEDI_features_Month_{month}',
        fileFormat='KML'    # CSV or KML
    )

    # Start the export task
    task.start()

# Process GEDI data for each month
for month in target_months:
    print(f"Processing GEDI data for Month {month}...")

    # Define time range (handle December separately)
    begin_date = ee.Date(f'{target_year}-{month:02d}-01')
    end_date   = ee.Date(f'{target_year}-{month+1:02d}-01') if month < 12 else ee.Date(f'{target_year+1}-01-01')

    # print(f'{target_year}-{month:02d}-01', f'{target_year}-{month+1:02d}-01')

    cdl_begin_date = ee.Date(f'{target_year}-01-01')
    cdl_end_date   = ee.Date(f'{target_year+1}-01-01')

    cdl = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(cdl_begin_date, cdl_end_date)).first()
    cropLandcover = cdl.select('cropland')
    cdl_composite = cropLandcover.reduce(ee.Reducer.mode())

    # Crop class masking
    maize = cdl_composite.eq(1)     # corn
    others = cdl_composite.neq(1)   # other crops + landcover

    # Cropland Masking
    dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
    target_dw_images = dw.filterDate(cdl_begin_date, cdl_end_date)
    classification = target_dw_images.select("label")
    dwComposite = classification.reduce(ee.Reducer.mode())
    cropland_lc = dwComposite.eq(4)

    maize_cropland_masked = maize.updateMask(cropland_lc).selfMask()
    others_cropland_masked = others.updateMask(cropland_lc).selfMask()

    maize_masked = maize_cropland_masked.clip(roi)
    others_masked = others_cropland_masked.clip(roi)

    gedi = (ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY').filter(ee.Filter.date(begin_date, end_date)).filter(ee.Filter.calendarRange(ee.Number(int(month)), ee.Number(int(month)), 'month'))
        .map(qualityMask)
        .map(sensitivity_filter)
        .select('rh.*')  # Select all RH bands
    )

    # Compute mean of selected data
    gedi_mean = gedi.mean().clip(roi)

    # Apply a mask using RH98
    gedi_98 = gedi_mean.select('rh98')
    gedi_mask = gedi_98.gt(0)
    gedi_masked = gedi_mean.updateMask(gedi_mask)

    # Apply the mask to each crop dataset
    gedi_crops = {
        "maize": maize_masked.updateMask(gedi_mask).selfMask(),
        "non-maize": others_masked.updateMask(gedi_mask).selfMask()
    }

    # Process all crops without predefined ROI, using image footprint as the boundary
    for crop_name, gedi_variable in gedi_crops.items():
        print(f" - Processing {crop_name} for Month {month}")
        process_gedi_corn(crop_name, gedi_variable, number_of_points, gedi_masked, month)

print("Export tasks started for all crops and months.")